# STM32G071RB LVGL Processing

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def add_transition_points(time, values):
    new_time = [time[0]]
    new_values = [values[0]]

    for i in range(1, len(time)):
        if values[i] != values[i - 1]:
            # Insert a point at the transition
            new_time.append(time[i])
            new_values.append(values[i - 1])
        new_time.append(time[i])
        new_values.append(values[i])

    return new_time, new_values

def find_edges(data):
    first_edge   = False
    lv_start_idx = 0
    lv_end_idx   = 0
    for idx, val in enumerate(data):
        if val == 1 and not first_edge:
            lv_start_idx = idx
            first_edge = True
        if val == 0 and first_edge:
            lv_end_idx = idx
            break
        
    if lv_start_idx is None or lv_end_idx is None:
        lv_start_idx = lv_end_idx = 0
        
    return (lv_start_idx, lv_end_idx)

def clear_file(filepath='README.md'):
    with open(filepath, 'w') as file:
        file.write('')

def write_file(lines, filepath='README.md'):
    with open(filepath, 'a') as file:
        for line in lines:
            print(line)
            file.write(line + '\n')

In [ ]:
clear_file()
write_file(["# STM32G071RB Measurements"])

# Screen Blit, Single Buffering, GCC Os

In [ ]:
title = "Screen Blit, Single Buffering, GCC Os"
path = 'saleae/screen_blit_os_sbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/screen_blit_os_sbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)

# Screen Blend, Single Buffering, GCC Os

In [ ]:
title = "Screen Blend, Single Buffering, GCC Os"
path = 'saleae/screen_blend_os_sbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/screen_blend_os_sbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)

# Aircon Main, Double Buffering, GCC Os

In [ ]:
title = "Aircon Main, Double Buffering, GCC Os"
path = 'saleae/aircon_main_os_dbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/aircon_main_os_dbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)

# Aircon Main, Single Buffering, GCC Os

In [ ]:
title = "Aircon Main, Single Buffering, GCC Os"
path = 'saleae/aircon_main_os_sbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/aircon_main_os_sbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)

# Aircon About, Single Buffering, GCC Os

In [ ]:
title = "Aircon About, Single Buffering, GCC Os"
path = 'saleae/aircon_about_os_sbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/aircon_about_os_sbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)

# Screen Blit, Single Buffering, GCC O3

In [ ]:
title = "Screen Blit, Single Buffering, GCC O3"
path = 'saleae/screen_blit_o3_sbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/screen_blit_o3_sbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)

# Screen Blend, Single Buffering, GCC O3

In [ ]:
title = "Screen Blend, Single Buffering, GCC O3"
path = 'saleae/screen_blend_o3_sbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/screen_blend_o3_sbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)

# Aircon Main, Double Buffering, GCC O3

In [ ]:
title = "Aircon Main, Double Buffering, GCC O3"
path = 'saleae/aircon_main_o3_dbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/aircon_main_o3_dbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)

# Aircon Main, Single Buffering, GCC O3

In [ ]:
title = "Aircon Main, Single Buffering, GCC O3"
path = 'saleae/aircon_main_o3_sbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/aircon_main_o3_sbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)

# Aircon About, Single Buffering, GCC O3

In [ ]:
title = "Aircon About, Single Buffering, GCC O3"
path = 'saleae/aircon_about_o3_sbuffering.csv'
data = pd.read_csv(path)

time                = data['Time [s]'].tolist()
refr_invalid_areas  = data['refr_invalid_areas'].tolist()
lv_timer_handler    = data['lv_timer_handler'].tolist()
flush_cb            = data['flush_cb'].tolist()

time_0, refr_invalid_areas_new    = add_transition_points(time, refr_invalid_areas)
time_1, lv_timer_handler_new      = add_transition_points(time, lv_timer_handler)
time_2, flush_cb_new              = add_transition_points(time, flush_cb)

fig, axs = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axs[0].plot(time_1, lv_timer_handler_new, linewidth=2, color='black')
axs[1].plot(time_0, refr_invalid_areas_new, linewidth=2, color='black')
axs[2].plot(time_2, flush_cb_new, linewidth=2, color='black')

start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
axs[0].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[0].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(refr_invalid_areas)
axs[1].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[1].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

start_idx, end_idx = find_edges(flush_cb)
print(start_idx)
axs[2].hlines(y=0.5, xmin=time[start_idx], xmax=time[end_idx], colors='blue', linestyles='--', linewidth=3)
axs[2].text((time[start_idx] + time[end_idx])/2, 0.6, f'{np.round((time[end_idx] - time[start_idx])*1000, 2)} ms', fontsize=12,
            bbox=dict(facecolor='white', edgecolor='blue', boxstyle='round,pad=0.3'))

axs[0].set_xlim(start_t, end_t)
axs[1].set_xlim(start_t, end_t)
axs[2].set_xlim(start_t, end_t)

fig.suptitle(title)
axs[0].set_title('lv_timer_handler')
axs[1].set_title('refr_invalid_areas')
axs[2].set_title('flush_cb')

axs[2].set_xlabel('time [s]')

fig.tight_layout()
fig.savefig('waveforms/aircon_about_o3_sbuffering.png')

In [ ]:
start_idx, end_idx = find_edges(lv_timer_handler)
start_t = time[start_idx] - 0.01
end_t   = time[end_idx] + 0.01
lv_timer_handler_t = time[end_idx] - time[start_idx]

start_idx, end_idx = find_edges(refr_invalid_areas)
refr_invalid_areas_t = time[end_idx] - time[start_idx]

flush_cb_filt = [flush_cb[i] for i in range(len(time)) if start_t <= time[i] <= end_t]
time_filt     = [time[i] for i in range(len(time)) if start_t <= time[i] <= end_t]

high    = False
sum     = 0
start   = 0
end     = 0
for idx, val in enumerate(flush_cb_filt):
    if val == 1 and not high:
        start = idx
        high = True
    if val == 0 and high:
        end  = idx
        sum += time_filt[end] - time_filt[start]
        high = False

flush_cb_t      = sum
render_time_t   = refr_invalid_areas_t - flush_cb_t

lines = [
    f"\n## {title}\n",
    f"lv_timer_handler: {np.round(lv_timer_handler_t*1000, 2)} ms",
    f"refr_invalid_areas: {np.round(refr_invalid_areas_t*1000, 2)} ms",
    f"flush_cb: {np.round(flush_cb_t*1000, 2)} ms",
    f"render_time: {np.round(render_time_t*1000, 2)} ms",
]

write_file(lines)